## Running Assessment for all SQL Servers

- Using Central Management Server
- Using [DBTOOLS funciton](https://www.powershellgallery.com/packages/dbatools/0.9.196/Content/functions%5CGet-DbaRegisteredServer.ps1)
- Saving result to database table
- Purge resutl based on retention

In [ ]:
# Use Group switch if you only want to run on certain groups
# See the function for other switches
$DBServers = Get-DbaRegisteredServer -SqlInstance CMSServer -Group DEV

  Foreach ($Server in $DBServers)
  {
      Write-host -nonewline "."
      #-- Make Sure Server is available before connecting
      $SQLManagementInstance = "ServerName where you want the result to be saved"
   
      Get-SqlInstance -ServerInstance $($Server.ServerName) | 
      Invoke-SqlAssessment -FlattenOutput |

Write-SqlTableData -ServerInstance $SQLManagementInstance -DatabaseName SQLAssessmentDemo -SchemaName Assessment -TableName Results -Force

 $Query=@"
IF OBJECT_ID('SQLAssessmentDemo..Assessment.Results') IS NULL
CREATE TABLE [Assessment].[Results](
	[CheckName] [nvarchar](max) NOT NULL,
	[CheckId] [nvarchar](max) NOT NULL,
	[RulesetName] [nvarchar](max) NOT NULL,
	[RulesetVersion] [nvarchar](max) NOT NULL,
	[Severity] [nvarchar](max) NOT NULL,
	[Message] [nvarchar](max) NOT NULL,
	[TargetPath] [nvarchar](max) NOT NULL,
	[TargetType] [nvarchar](max) NOT NULL,
	[HelpLink] [nvarchar](max) NOT NULL,
	[Timestamp] [datetimeoffset](7) NOT NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]



IF OBJECT_ID('SQLAssessmentDemo..Assessment.Results_Archive') IS NULL
CREATE TABLE [Assessment].[Results_Archive](
	[CheckName] [nvarchar](max) NOT NULL,
	[CheckId] [nvarchar](max) NOT NULL,
	[RulesetName] [nvarchar](max) NOT NULL,
	[RulesetVersion] [nvarchar](max) NOT NULL,
	[Severity] [nvarchar](max) NOT NULL,
	[Message] [nvarchar](max) NOT NULL,
	[TargetPath] [nvarchar](max) NOT NULL,
	[TargetType] [nvarchar](max) NOT NULL,
	[HelpLink] [nvarchar](max) NOT NULL,
	[Timestamp] [datetimeoffset](7) NOT NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]

INSERT INTO [Assessment].[Results]
SELECT * FROM [Assessment].[Results]
--Adjust result retention period
DELETE FROM [Assessment].[Results_Archive] WHERE RecCreatedDt< GETDATE()-7

TRUNCATE TABLE SQLAssessmentDemo..[Assessment].[Results]
"@

}